In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import random

In [ ]:
with open("mnist_data/X.npy","rb") as f:
    x_train = np.load(f)
    
with open("mnist_data/y.npy","rb") as f:
    y_train = np.load(f)



In [ ]:
image_no = random.randint(0,x_train.shape[0])
plt.imshow(x_train[image_no].reshape(20,20),cmap="gray")
y_train[image_no].item()

In [ ]:
x_train = torch.from_numpy(x_train)
y_train = torch.tensor(y_train,dtype=torch.long)

In [ ]:
x_train.dtype

In [ ]:
class DigitNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(400,25)
        self.l2 = nn.Linear(25,15)
        self.l3 = nn.Linear(15,10)
        self.act = nn.ReLU()
    
    def forward(self,x):
        x = self.act(self.l1(x))
        x = self.act(self.l2(x))
        x = self.l3(x)
        return x

In [ ]:
def train(x,y,epochs,lr,model):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr)
    p_bar = tqdm(total=epochs, desc="Training", unit="epoch")
    for _ in range(epochs):
        optimizer.zero_grad()
        loss = loss_fn(model(x),y.reshape(y.shape[0],))
        loss.backward()
        optimizer.step()
        p_bar.set_postfix({"Loss": loss.item()})
        p_bar.update(1)
    p_bar.close()


In [ ]:
def predict(x,y,model):
    i = random.randint(0,x.shape[0])
    _,index = torch.max(model(x[i].reshape(1,-1)),dim=1)
    plt.title(f"Image no:{i} Label:{y[i].item()}  Prediction:{index.item()}")
    plt.imshow(x[i].reshape(20,20),cmap="gray")

In [ ]:
model = DigitNet().double()

In [ ]:
model

In [ ]:
train(x_train,y_train,1500,0.01,model)

In [ ]:
predict(x_train,y_train,model)

In [ ]:
m, n = x_train.shape

fig, axes = plt.subplots(8,8, figsize=(8,8))
fig.tight_layout(pad=0.1,rect=[0, 0.03, 1, 0.92]) #[left, bottom, right, top]

for i,ax in enumerate(axes.flat):
    # Select random indices
    random_index = np.random.randint(m)
    
    # Select rows corresponding to the random indices and
    # reshape the image
    X_random_reshaped = x_train[random_index].reshape((20,20)).T
    
    # Display the image
    ax.imshow(X_random_reshaped, cmap='gray')
    
    # Predict using the Neural Network
    _,index = torch.max(model(x_train[random_index].reshape(1,-1)),dim=1)
    yhat = index.item()
        
    # Display the label above the image
    ax.set_title(f"{int(y_train[random_index,0])},{yhat}")
    ax.set_axis_off()
fig.suptitle("Label, yhat", fontsize=16)
plt.show()